Final NTFF implementation given update eqns

In [ ]:
#TESTING ONLY
# Importing necessary variables for testing only purposes
import matplotlib.pyplot as plt
import math, cmath
import numpy as np

# Simulation Parameters
e0 = 1        
mu0 = 1       
c0 = 1 / math.sqrt(e0 * mu0) 
dx = 0.5e-3 
dy = 0.5e-3 
magic_time_step = dx / (np.sqrt(2) * c0)
max_time_steps = 500

# Grid in cell units
whole_grid = 100e-3 # whole grid = 30 mm
numX_cells = int(whole_grid / dx)  
numY_cells = int(whole_grid / dy)   
center_x = numX_cells // 2
center_y = numY_cells // 2

# Set up the permittivity grid 
grid = np.ones((numY_cells, numX_cells)) * e0

# Source locations
dipole_spacing_cells = int(15e-3 / dx) # cells between dipoles
x1_source = center_x - (dipole_spacing_cells //2)
x2_source = center_x + (dipole_spacing_cells//2)
y1_source = center_y
y2_source = center_y

# Source parameters
lambda0 = 6e-2 # wavelength for 5G is 6 cm
lambda_lower = 5e-2
lambda_upper = 7e-2
omega0 = (2 * np.pi * c0) / lambda0
sigma = (2 / omega0) * (lambda0 / (lambda_upper - lambda_lower))

# Arrays to store fields
Ez = np.zeros((numX_cells, numY_cells))
Hx = np.zeros((numX_cells, numY_cells))
Hy = np.zeros((numX_cells, numY_cells))

In [ ]:
#TESTING ONLY
#dipole simulation
# Dipole Simulation

e_dipole = 100 # permitivity of copper is close to infinity
dipole_width = 1e-3 # dipole thickness = 1 mm
dipole_height = 15e-3 # dipole height = 15 mm
dipole_width_cells = int(dipole_width / dx)         
dipole_height_cells = int(dipole_height / dy)  

# Dipole locations
dipole1_center = x1_source
dipole1_left = dipole1_center - (dipole_width_cells // 2)
dipole1_right = dipole1_left + dipole_width_cells - 1
dipole1_bottom = center_y - (dipole_height_cells // 2)
dipole1_top = dipole1_bottom + dipole_height_cells - 1

dipole2_center = x2_source
dipole2_left = dipole2_center - (dipole_width_cells // 2)
dipole2_right = dipole2_left + dipole_width_cells - 1
dipole2_bottom = center_y - (dipole_height_cells // 2)
dipole2_top = dipole2_bottom + dipole_height_cells - 1

# Place left dipole in permittivity grid
for x in range(dipole1_left, dipole1_right + 1):
    for y in range(dipole1_bottom, dipole1_top + 1):
        grid[y, x] = e_dipole

# Place right dipole in permittivity grid
for x in range(dipole2_left, dipole2_right + 1):
    for y in range(dipole2_bottom, dipole2_top + 1):
        grid[y, x] = e_dipole

In [3]:
#Defining closed contours
import numpy as np
#defining square boundary form L1 --> L4
line_size=80e-3 #each line is 80mm
numL_cells=int(line_size // dx)

L_starting_dim= 10e-3 #starting index for L2, L4
L_starting_cell_idx = int(L_starting_dim //dx)
L_ending_dim=90e-3
L_ending_cell_idx=L_starting_cell_idx + numL_cells

#L1 is vertical, RHS line
L1_col1 = np.full(numL_cells, L_ending_cell_idx)
L1_col2=np.arange(L_starting_cell_idx, L_ending_cell_idx)

L1_idxs = np.column_stack((L1_col1,L1_col2))

#L2 is vertical, LHS line
L2_col1=np.full(numL_cells,L_starting_cell_idx)
L2_col2=np.arange(L_starting_cell_idx, L_ending_cell_idx)
L2_idxs=np.column_stack((L2_col1,L2_col2))

#L3 is horizontal, top line
L3_col1=np.arange(L_starting_cell_idx, L_ending_cell_idx)
L3_col2=np.full(numL_cells, L_ending_cell_idx)
L3_idxs=np.column_stack((L3_col1,L3_col2))

#L4 is horizontal, bottom line
L4_col1=np.arange(L_starting_cell_idx, L_ending_cell_idx)
L4_col2=np.full(numL_cells, L_starting_cell_idx)
L4_idxs=np.column_stack((L4_col1,L4_col2))

#print(L1_idxs)

In [ ]:
# TESTING ONLY
# Running FDTD for testing purposes only
# 2D FDTD Model

# Using absorbing boundary condition to simulate the energy being absorbed in air.
boundaryLeft = np.zeros(numY_cells)
boundaryRight = np.zeros(numY_cells)
boundaryBottom = np.zeros(numX_cells)
boundaryTop = np.zeros(numX_cells)

#defining Ez_time_L1
Ez_time_L1 = np.zeros((numL_cells, max_time_steps))
Hx_time_L1 = np.zeros((numL_cells, max_time_steps))
Hy_time_L1 = np.zeros((numL_cells, max_time_steps))

#defining L2
Ez_time_L2 = np.zeros((numL_cells, max_time_steps))
Hx_time_L2 = np.zeros((numL_cells, max_time_steps))
Hy_time_L2 = np.zeros((numL_cells, max_time_steps))

#defining L3
Ez_time_L3 = np.zeros((numL_cells, max_time_steps))
Hx_time_L3 = np.zeros((numL_cells, max_time_steps))
Hy_time_L3 = np.zeros((numL_cells, max_time_steps))

#defining L4
Ez_time_L4 = np.zeros((numL_cells, max_time_steps))
Hx_time_L4 = np.zeros((numL_cells, max_time_steps))
Hy_time_L4 = np.zeros((numL_cells, max_time_steps))

#combining time domain into lists
L_idxs_list = [L1_idxs, L2_idxs, L3_idxs, L4_idxs]
Ez_time_list = [Ez_time_L1, Ez_time_L2, Ez_time_L3, Ez_time_L4]
Hx_time_list = [Hx_time_L1, Hx_time_L2, Hx_time_L3, Hx_time_L4]
Hy_time_list = [Hy_time_L1, Hy_time_L2, Hy_time_L3, Hy_time_L4]

# Main 2D FDTD Loop
for n in range(max_time_steps):

    # Compute Hy 
    for i in range(numX_cells - 1):
        for j in range(numY_cells):
            Hy[i, j] += (Ez[i + 1, j] - Ez[i, j]) * magic_time_step / dx
            for k in range(4):
                match = np.any((L_idxs_list[k][:, 0] == i) & (L_idxs_list[k][:, 1] == j))
                if match:
                    idx = np.where((L_idxs_list[k][:, 0] == i) & (L_idxs_list[k][:, 1] == j))[0]
                    Hy_time_list[k][idx, n] = Hy[i, j]



     # Compute Hx
    for i in range(numX_cells):
        for j in range(numY_cells - 1):
            Hx[i, j] -= (Ez[i, j + 1] - Ez[i, j]) * magic_time_step / dy
            for k in range(4):
                match = np.any((L_idxs_list[k][:, 0] == i) & (L_idxs_list[k][:, 1] == j))
                if match:
                    idx = np.where((L_idxs_list[k][:, 0] == i) & (L_idxs_list[k][:, 1] == j))[0]
                    Hx_time_list[k][idx, n] = Hx[i, j]


    # Use temp variables to store left and bottom Ez values sorta like a moving window; only 2 sides needed bc symmetrical
    tempLeft = Ez[1, :].copy() 
    tempBottom = Ez[:, 1].copy()  

    # Compute Ez
    for i in range(1, numX_cells - 1):
        for j in range(1, numY_cells - 1):
            Ez[i, j] += ((Hy[i, j] - Hy[i - 1, j]) * magic_time_step / dx) - ((Hx[i, j] - Hx[i, j - 1]) * magic_time_step / dy)

            #recording Ez at L1
            for k in range(4):
                match = np.any((L_idxs_list[k][:, 0] == i) & (L_idxs_list[k][:, 1] == j))
                if match:
                    idx = np.where((L_idxs_list[k][:, 0] == i) & (L_idxs_list[k][:, 1] == j))[0]
                    Ez_time_list[k][idx, n] = Ez[i, j]


            
    # Apply excitation at left source
    t_n = n * magic_time_step
    Ez[x1_source, y1_source] += np.exp(-((t_n - 4 * sigma) ** 2) / sigma ** 2) * np.sin(omega0 * t_n)

    # Apply excitation at right source
    t_n = n * magic_time_step
    Ez[x2_source, y2_source] += np.exp(-((t_n - 4 * sigma) ** 2) / sigma ** 2) * np.sin(omega0 * t_n)

    # Update PEC conditions
    Ez[0, :] = boundaryLeft
    Ez[-1, :] = boundaryRight
    Ez[:, 0] = boundaryBottom
    Ez[:, -1] = boundaryTop

    # Update boundary buffers for next time step
    boundaryLeft = tempLeft
    boundaryBottom = tempBottom
    boundaryRight = Ez[-2, :].copy()
    boundaryTop   = Ez[:, -2].copy()
    
    # Visualize Ez field every 50 time steps
    if n % 50 == 0:
        plt.clf() 
        plt.imshow(Ez, cmap='viridis', origin='lower')
        plt.title(f"Time Step {n}")
        plt.xlabel("X")
        plt.ylabel("Y")
        plt.colorbar(label="Ez")
        plt.pause(0.01) 

print(Ez_time_L1)
plt.show()

In [ ]:
#Performing FFT along closed contours
Ez_t_list = [Ez_time_L1, Ez_time_L2, Ez_time_L3, Ez_time_L4]

Ez_fft_vals = [np.fft.fft(Ez_time, axis=1) for Ez_time in Ez_t_list]
Ez_freqs = [np.fft.fftfreq(max_time_steps, d=magic_time_step) for _ in Ez_t_list]


Hx_t_list = [Hx_time_L1, Hx_time_L2, Hx_time_L3, Hx_time_L4]

Hx_fft_vals = [np.fft.fft(Hx_time, axis=1) for Hx_time in Hx_t_list]
Hx_freqs = [np.fft.fftfreq(max_time_steps, d=magic_time_step) for _ in Hx_t_list]


Hy_t_list = [Hy_time_L1, Hy_time_L2, Hy_time_L3, Hy_time_L4]

Hy_fft_vals = [np.fft.fft(Hy_time, axis=1) for Hy_time in Hy_t_list]
Hy_freqs = [np.fft.fftfreq(max_time_steps, d=magic_time_step) for _ in Hy_t_list]


peak_freq_val=16.970562748477143

In [ ]:
#Computing equivalent J and M current densities
for freqs_arr in Ez_freqs:

    target_freq= omega0/(2*np.pi)
    bin_idx = np.argmin(np.abs(freqs_arr - target_freq))
    print('Bin index is ',bin_idx)


Meq_L1_phasors = Ez_fft_vals[0][:,bin_idx]
Meq_L2_phasors = -Ez_fft_vals[1][:,bin_idx]
Meq_L3_phasors = -Ez_fft_vals[2][:,bin_idx]
Meq_L4_phasors= Ez_fft_vals[3][:,bin_idx]

Jeq_L1_phasors=Hy_fft_vals[0][:,bin_idx]
Jeq_L2_phasors=-Hy_fft_vals[1][:,bin_idx]
Jeq_L3_phasors=-Hx_fft_vals[2][:,bin_idx]
Jeq_L4_phasors=Hx_fft_vals[3][:,bin_idx]

#creating list for all Jeq and Meq components along grid
Meq_phasors=[Meq_L1_phasors,Meq_L2_phasors,Meq_L3_phasors,Meq_L4_phasors]
Jeq_phasors=[Jeq_L1_phasors,Jeq_L2_phasors,Jeq_L3_phasors,Jeq_L4_phasors]

In [ ]:
#Computing r' cos (phi) for each exponential factor in scattering width eqn

#Nested loop to compute each individual line integral component, for each theta

#Computing scattering width for each theta component

theta_input_param=np.linspace(1,360,360)
r_prime_cos_phi_vec=[]
sigma_2d=np.zeros(360)

#replace lambda0 with lambda
lambda_new=c0/peak_freq_val
k = 2 * np.pi / lambda_new
E_initial_amplitude = 1
for theta in theta_input_param:
    theta_deg = theta
    r_prime_cos_phi_vec_L1=[]
    r_prime_vec_L1=[]
    phi_vector_L1=[]

    theta_rad=math.radians(theta_deg)
    r_mag_L1=((L_ending_cell_idx-center_x)*dx) / math.cos(theta_rad)
    for x, y in L1_idxs:
        x_y_distances=np.array([(x-center_x)*dx,(y-center_y)*dy])
        r_prime_mag=np.linalg.norm(x_y_distances)
        
        theta_prime_rad = math.asin ( ((y-center_y)*dy) / r_prime_mag)
        theta_prime_deg=math.degrees(theta_prime_rad)
        phi=theta_deg-theta_prime_deg

        phi_rad=math.radians(phi)
        phi_vector_L1.append(phi)

        phase_const= r_mag_L1 * math.cos(phi_rad)

        r_prime_cos_phi_vec_L1.append(phase_const)
    
    r_prime_cos_phi_vec_L2=[]
    r_prime_vec_L2=[]
    phi_vector_L2=[]
    r_mag_L2=abs(((L_starting_cell_idx-center_x)*dx)) / math.cos(theta_rad)

    for x, y in L2_idxs:
        x_y_distances=np.array([(x-center_x)*dx,(y-center_y)*dy])
        r_prime_mag=abs(np.linalg.norm(x_y_distances))
        
        theta_prime_rad = math.asin ( ((y-center_y)*dy) / r_prime_mag)
        theta_prime_deg=math.degrees(theta_prime_rad)
        phi=-((theta_deg-180)-theta_prime_deg)

        phi_rad=math.radians(phi)
        phi_vector_L2.append(phi)

        phase_const= r_mag_L2 * math.cos(phi_rad)

        r_prime_cos_phi_vec_L2.append(phase_const)

    r_prime_cos_phi_vec_L3=[]
    r_prime_vec_L3=[]
    phi_vector_L3=[]
    r_mag_L3=abs(((L_ending_cell_idx-center_y*dy) / math.sin(theta_rad)))

    for x, y in L3_idxs:
        x_y_distances=np.array([(x-center_x)*dx,(y-center_y)*dy])
        r_prime_mag=abs(np.linalg.norm(x_y_distances))
        
        theta_prime_rad = math.acos ( ((x-center_x)*dx) / r_prime_mag)
        theta_prime_deg=math.degrees(theta_prime_rad)
        phi=(180-theta_deg)-theta_prime_deg

        phi_rad=math.radians(phi)
        phi_vector_L3.append(phi)

        phase_const= r_mag_L3 * math.cos(phi_rad)

        r_prime_cos_phi_vec_L3.append(phase_const)

   
    r_prime_cos_phi_vec_L4=[]
    r_prime_vec_L4=[]
    phi_vector_L4=[]
    r_mag_L4=abs(((L_starting_cell_idx-center_y*dy) / math.sin(theta_rad)))

    for x, y in L4_idxs:
        x_y_distances=np.array([(x-center_x)*dx,(y-center_y)*dy])
        r_prime_mag=abs(np.linalg.norm(x_y_distances))
        
        theta_prime_rad = math.acos ( ((x-center_x)*dx) / r_prime_mag)
        theta_prime_deg=math.degrees(theta_prime_rad)
        phi=-((theta_deg-180)-theta_prime_deg)

        phi_rad=math.radians(phi)
        phi_vector_L4.append(phi)

        phase_const= r_mag_L4 * math.cos(phi_rad)

        r_prime_cos_phi_vec_L4.append(phase_const)

    r_cos_factors=[r_prime_cos_phi_vec_L1,r_prime_cos_phi_vec_L2,r_prime_cos_phi_vec_L3,r_prime_cos_phi_vec_L4]
    r_cos_factors=np.array(r_cos_factors)

#want to check if getting same plot when only considering L1
    #L_integral_arr=[ ((omega0*mu0*np.sum(Jeq[0:-1])) - (k*Meq[0:-1]))*np.exp(1j*k*r_cos[0:-1])
                #for Jeq, Meq, r_cos in zip(Jeq_phasors,Meq_phasors, r_cos_factors)]
    L_integral_arr= ((omega0*mu0*np.sum(Jeq_phasors[2][0:-1])) - (k*Meq_phasors[2][0:-1]))*np.exp(1j*k*r_cos_factors[2][0:-1])
                #for Jeq, Meq, r_cos in zip(Jeq_phasors,Meq_phasors, r_cos_factors)]
   
    #L_integral_vals=[ (np.sum(contour_val)) for contour_val in L_integral_arr]
    L_integral_vals= (np.sum(contour_val) for contour_val in L_integral_arr)

    L_integral_final=(abs(np.sum(L_integral_vals)))**2
    sigma_2d[int(theta) - 1]=(lambda_new / (8*np.pi*(E_initial_amplitude**2))) * L_integral_final



In [ ]:
#Plotting Scattering width

 
plt.figure()
plt.plot(theta_input_param, sigma_2d)
plt.xlabel('Theta (degrees)')
plt.ylabel('Scattering width')
plt.title('NTFF for given theta')
plt.show()